In [1]:
# Install required libraries
!pip install yfinance PyPortfolioOpt


  Using cached pyportfolioopt-1.5.5-py3-none-any.whl.metadata (23 kB)
  Using cached clarabel-0.9.0-cp37-abi3-win_amd64.whl.metadata (4.8 kB)
Using cached pyportfolioopt-1.5.5-py3-none-any.whl (61 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 4.8 MB/s eta 0:00:00
Using cached clarabel-0.9.0-cp37-abi3-win_amd64.whl (736 kB)
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/8.4 MB 5.6 MB/s eta 0:00:02
   --------- ------------------------------ 2.1/8.4 MB 5.6 MB/s eta 0:00:02
   -------------- ------------------------- 3.1/8.4 MB 5.1 MB/s eta 0:00:02
   ------------------- -------------------- 4.2/8.4 MB 5.2 MB/s eta 0:00:01
   ------------------------ --------------- 5.2/8.4 MB 5.1 MB/s eta 0:00:01
   ---------------------------- ----------- 6.0/8.4 MB 4.8 MB/s eta 0:00:01

In [2]:
#Importing Libs



import yfinance as yf
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.expected_returns import mean_historical_return
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#Define Stock Tickers and Fetch Data




# Define Indian stock tickers
tickers = ['INFY.NS', 'HDFCBANK.NS', 'SUNPHARMA.NS', 'RELIANCE.NS', 'BHARTIARTL.NS']

# Fetch historical adjusted closing prices
data = yf.download(tickers, start="2020-01-01", end="2023-01-01")['Adj Close']

# Display the first few rows of the dataset
data.head()


[*********************100%***********************]  5 of 5 completed


Ticker,BHARTIARTL.NS,HDFCBANK.NS,INFY.NS,RELIANCE.NS,SUNPHARMA.NS
Date,,,,,
2020-01-01 00:00:00+00:00,436.938934,1227.428589,655.615234,672.908081,412.528442
2020-01-02 00:00:00+00:00,438.770355,1235.252441,653.702271,684.364075,413.145874
2020-01-03 00:00:00+00:00,438.673981,1217.636841,663.756531,685.188721,422.312164
2020-01-06 00:00:00+00:00,433.420685,1191.285522,657.394958,669.297607,417.895233
2020-01-07 00:00:00+00:00,429.034912,1210.148926,647.652161,679.594482,424.021851


In [4]:
#Calculate Expected Returns and Covariance Matrix




# Calculate expected returns and covariance matrix
mu = mean_historical_return(data)
S = CovarianceShrinkage(data).ledoit_wolf()

# Display calculated values
print("Expected Returns:\n", mu)
print("\nCovariance Matrix:\n", S)


Expected Returns:
 Ticker
BHARTIARTL.NS    0.225623
HDFCBANK.NS      0.090900
INFY.NS          0.302098
RELIANCE.NS      0.204558
SUNPHARMA.NS     0.340268
dtype: float64

Covariance Matrix:
 Ticker         BHARTIARTL.NS  HDFCBANK.NS   INFY.NS  RELIANCE.NS  SUNPHARMA.NS
Ticker                                                                        
BHARTIARTL.NS       0.106107     0.036635  0.028873     0.032506      0.029354
HDFCBANK.NS         0.036635     0.097708  0.037135     0.046949      0.026508
INFY.NS             0.028873     0.037135  0.094080     0.039100      0.027641
RELIANCE.NS         0.032506     0.046949  0.039100     0.118068      0.032405
SUNPHARMA.NS        0.029354     0.026508  0.027641     0.032405      0.089477


In [5]:
#Optimize Portfolio Allocation




# Create the Efficient Frontier object
ef = EfficientFrontier(mu, S)

# Optimize for a target risk level
weights = ef.efficient_risk(target_volatility=0.36)

# Clean weights to round small values
cleaned_weights = ef.clean_weights()

# Display portfolio weights
pd.DataFrame.from_dict(cleaned_weights, orient='index', columns=['Weight']).style.background_gradient(cmap="YlGn")


,Weight
BHARTIARTL.NS,0.000000
HDFCBANK.NS,0.000000
INFY.NS,0.000000
RELIANCE.NS,0.000000
SUNPHARMA.NS,1.000000
